# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [3]:
import os
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
import math
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from imblearn.under_sampling import RandomUnderSampler


if 'PG_PORT' in os.environ:

    host = 'db'
    port = os.environ['PG_PORT']
    database = os.environ['PG_DATABASE']
    user = os.environ['PG_USER']
    password = os.environ['PG_PASSWORD']
    
    # pd.read_sql用のコネクタ
    conn = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

    df_customer = pd.read_sql(sql='select * from customer', con=conn)
    df_category = pd.read_sql(sql='select * from category', con=conn)
    df_product = pd.read_sql(sql='select * from product', con=conn)
    df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
    df_store = pd.read_sql(sql='select * from store', con=conn)
    df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

else:
    if not os.path.exists('../data/'):
        !git clone https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess
        os.chdir('100knocks-preprocess/docker/work/answer')

    dtype = {
        'customer_id': str,
        'gender_cd': str,
        'postal_cd': str,
        'application_store_cd': str,
        'status_cd': str,
        'category_major_cd': str,
        'category_medium_cd': str,
        'category_small_cd': str,
        'product_cd': str,
        'store_cd': str,
        'prefecture_cd': str,
        'tel_no': str,
        'postal_cd': str,
        'street': str
    }
    
    df_customer = pd.read_csv("../data/customer.csv", dtype=dtype)
    df_category = pd.read_csv("../data/category.csv", dtype=dtype)
    df_product = pd.read_csv("../data/product.csv", dtype=dtype)
    df_receipt = pd.read_csv("../data/receipt.csv", dtype=dtype)
    df_store = pd.read_csv("../data/store.csv", dtype=dtype)
    df_geocode = pd.read_csv("../data/geocode.csv", dtype=dtype)


# 演習問題

---
> P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [4]:
df_receipt.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
5,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
6,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
7,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
8,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
9,20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


---
> P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [7]:
new = df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']]
print(new.head(10))

   sales_ymd     customer_id  product_cd  amount
0   20181103  CS006214000001  P070305012     158
1   20181118  CS008415000097  P070701017      81
2   20170712  CS028414000014  P060101005     170
3   20190205  ZZ000000000000  P050301001      25
4   20180821  CS025415000050  P060102007      90
5   20190605  CS003515000195  P050102002     138
6   20181205  CS024514000042  P080101005      30
7   20190922  CS040415000178  P070501004     128
8   20170504  ZZ000000000000  P071302010     770
9   20191010  CS027514000015  P071101003     680


---
> P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更して抽出すること。

In [8]:
new = df_receipt.rename(columns={'sales_ymd': 'sales_date'})[['sales_date', 'customer_id', 'product_cd', 'amount']]
print(new.head(10))

   sales_date     customer_id  product_cd  amount
0    20181103  CS006214000001  P070305012     158
1    20181118  CS008415000097  P070701017      81
2    20170712  CS028414000014  P060101005     170
3    20190205  ZZ000000000000  P050301001      25
4    20180821  CS025415000050  P060102007      90
5    20190605  CS003515000195  P050102002     138
6    20181205  CS024514000042  P080101005      30
7    20190922  CS040415000178  P070501004     128
8    20170504  ZZ000000000000  P071302010     770
9    20191010  CS027514000015  P071101003     680


---
> P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [9]:
select = df_receipt[df_receipt['customer_id'] == 'CS018205000001']
new = select[['sales_ymd', 'customer_id', 'product_cd', 'amount']]
print(new)

       sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
9843    20180414  CS018205000001  P060104007     600
21110   20170614  CS018205000001  P050206001     990
27673   20170614  CS018205000001  P060702015     108
27840   20190216  CS018205000001  P071005024     102
28757   20180414  CS018205000001  P071101002     278
39256   20190226  CS018205000001  P070902035     168
58121   20190924  CS018205000001  P060805001     495
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100
88508   20190216  CS018205000001  P040101002     218
91525   20190924  CS018205000001  P091503001     280


---
> P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [19]:
select = df_receipt[(df_receipt['customer_id'] == 'CS018205000001') & (df_receipt['amount'] >= 1000)]
new = select[['sales_ymd', 'customer_id', 'product_cd', 'amount']]
print(new)

       sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100


---
> P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [20]:
select = df_receipt[(df_receipt['customer_id'] == 'CS018205000001') & (df_receipt['amount'] >= 1000)| (df_receipt['quantity'] >= 5)]
new = select[['sales_ymd', 'customer_id', 'product_cd', 'amount']]
print(new)

       sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
9843    20180414  CS018205000001  P060104007     600
21110   20170614  CS018205000001  P050206001     990
47082   20190610  CS037414000080  P090401002     640
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100


---
> P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [21]:
select = df_receipt[(df_receipt['customer_id'] == 'CS018205000001') & (df_receipt['amount'] >= 1000)&(df_receipt['amount'] <= 2000)]
new = select[['sales_ymd', 'customer_id', 'product_cd', 'amount']]
print(new)

       sales_ymd     customer_id  product_cd  amount
72254   20180911  CS018205000001  P071401005    1100


---
> P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [22]:
select = df_receipt[
    (df_receipt['customer_id'] == 'CS018205000001') &
    (df_receipt['amount'] >= 1000) &
    (df_receipt['product_cd'] != 'P071401019')
]
new = select[['sales_ymd', 'customer_id', 'product_cd', 'amount']]
print(new)

       sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
> 
> `df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [23]:
df_store.query('prefecture_cd != "13" & floor_area <= 900')


,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0


---
> P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [25]:
select = df_store[df_store['store_cd'].str.startswith('S14')]
print(select.head(10))


   store_cd store_name prefecture_cd prefecture              address  \
2    S14010        菊名店            14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3    S14033       阿久和店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
4    S14036     相模原中央店            14       神奈川県        神奈川県相模原市中央二丁目   
7    S14040       長津田店            14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
9    S14050      阿久和西店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
12   S14028       二ツ橋店            14       神奈川県       神奈川県横浜市瀬谷区二ツ橋町   
16   S14012      本牧和田店            14       神奈川県        神奈川県横浜市中区本牧和田   
18   S14046       北山田店            14       神奈川県     神奈川県横浜市都筑区北山田一丁目   
19   S14022        逗子店            14       神奈川県         神奈川県逗子市逗子一丁目   
20   S14011      日吉本町店            14       神奈川県    神奈川県横浜市港北区日吉本町四丁目   

                      address_kana        tel_no  longitude  latitude  \
2        カナガワケンヨコハマシコウホククキクナイッチョウメ  045-123-4032   139.6326  35.50049   
3        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4043   139.4961  

---
> P-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [26]:
select = df_store[df_store['store_cd'].str.endswith('1')]
print(select.head(10))

   store_cd store_name prefecture_cd prefecture            address  \
5    S13051      板橋大原店            13        東京都          東京都板橋区大原町   
13   S13031        初台店            13        東京都        東京都渋谷区初台二丁目   
15   S13001       仲六郷店            13        東京都       東京都大田区仲六郷二丁目   
20   S14011      日吉本町店            14       神奈川県  神奈川県横浜市港北区日吉本町四丁目   
29   S13041       八王子店            13        東京都          東京都八王子市大塚   
36   S14021       伊勢原店            14       神奈川県     神奈川県伊勢原市伊勢原四丁目   

                      address_kana        tel_no  longitude  latitude  \
5               トウキョウトイタバシクオオハラチョウ  03-0123-4029   139.6980  35.76788   
13             トウキョウトシブヤクハツダイニチョウメ  03-0123-4020   139.6862  35.67616   
15           トウキョウトオオタクナカロクゴウニチョウメ  03-1234-4007   139.7132  35.55135   
20  カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ  045-123-4033   139.6316  35.54655   
29                トウキョウトハチオウジシオオツカ  042-123-4026   139.4235  35.63787   
36           カナガワケンイセハラシイセハラヨンチョウメ  046-123-4035   139.3129  35.40169  

---
> P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [28]:
select = df_store[df_store['address'].str.contains('横浜市')]
print(select.head(10))

   store_cd store_name prefecture_cd prefecture              address  \
2    S14010        菊名店            14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3    S14033       阿久和店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
7    S14040       長津田店            14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
9    S14050      阿久和西店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
12   S14028       二ツ橋店            14       神奈川県       神奈川県横浜市瀬谷区二ツ橋町   
16   S14012      本牧和田店            14       神奈川県        神奈川県横浜市中区本牧和田   
18   S14046       北山田店            14       神奈川県     神奈川県横浜市都筑区北山田一丁目   
20   S14011      日吉本町店            14       神奈川県    神奈川県横浜市港北区日吉本町四丁目   
26   S14048      中川中央店            14       神奈川県    神奈川県横浜市都筑区中川中央二丁目   
40   S14042       新山下店            14       神奈川県      神奈川県横浜市中区新山下二丁目   

                      address_kana        tel_no  longitude  latitude  \
2        カナガワケンヨコハマシコウホククキクナイッチョウメ  045-123-4032   139.6326  35.50049   
3        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4043   139.4961  

---
> P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [30]:
select = df_customer[df_customer['status_cd'].str.match(r'^[A-F]')]
print(select.head(10))

       customer_id customer_name gender_cd gender   birth_day  age postal_cd  \
2   CS031415000172       宇多田 貴美子         1     女性  1976-10-04   42  151-0053   
6   CS015414000103         奥野 陽子         1     女性  1977-08-09   41  136-0073   
12  CS011215000048         芦田 沙耶         1     女性  1992-02-01   27  223-0062   
15  CS029415000023         梅田 里穂         1     女性  1976-01-17   43  279-0043   
21  CS035415000029         寺沢 真希         9     不明  1977-09-27   41  158-0096   
32  CS031415000106        宇野 由美子         1     女性  1970-02-26   49  151-0053   
33  CS029215000025         石倉 美帆         1     女性  1993-09-28   25  279-0022   
40  CS033605000005         猪股 雄太         0     男性  1955-12-05   63  246-0031   
44  CS033415000229        板垣 菜々美         1     女性  1977-11-07   41  246-0021   
53  CS008415000145         黒谷 麻緒         1     女性  1977-06-27   41  157-0067   

                     address application_store_cd application_date  \
2        東京都渋谷区代々木**********               S13031

---
> P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [33]:
select = df_customer[df_customer['status_cd'].str.contains(r'[1-9]$')]
print(select.head(10))


       customer_id customer_name gender_cd gender   birth_day  age postal_cd  \
4   CS001215000145         田崎 美紀         1     女性  1995-03-29   24  144-0055   
9   CS033513000180          安斎 遥         1     女性  1962-07-11   56  241-0823   
12  CS011215000048         芦田 沙耶         1     女性  1992-02-01   27  223-0062   
14  CS040412000191         川井 郁恵         1     女性  1977-01-05   42  226-0021   
16  CS009315000023         皆川 文世         1     女性  1980-04-15   38  154-0012   
22  CS015315000033        福士 璃奈子         1     女性  1983-03-17   36  135-0043   
23  CS023513000066         神戸 そら         1     女性  1961-12-17   57  210-0005   
24  CS035513000134         市川 美帆         1     女性  1960-03-27   59  156-0053   
27  CS001515000263         高松 夏空         1     女性  1962-11-09   56  144-0051   
28  CS040314000027       鶴田 きみまろ         9     不明  1986-03-26   33  226-0027   

                     address application_store_cd application_date  \
4        東京都大田区仲六郷**********               S13001

---
> P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [34]:
select = df_customer[df_customer['status_cd'].str.match(r'^[A-F].*[1-9]$')]
print(select.head(10))


        customer_id customer_name gender_cd gender   birth_day  age postal_cd  \
12   CS011215000048         芦田 沙耶         1     女性  1992-02-01   27  223-0062   
68   CS022513000105        島村 貴美子         1     女性  1962-03-12   57  249-0002   
71   CS001515000096         水野 陽子         9     不明  1960-11-29   58  144-0053   
122  CS013615000053         西脇 季衣         1     女性  1953-10-18   65  261-0026   
144  CS020412000161          小宮 薫         1     女性  1974-05-21   44  174-0042   
178  CS001215000097        竹中 あさみ         1     女性  1990-07-25   28  146-0095   
252  CS035212000007        内村 恵梨香         1     女性  1990-12-04   28  152-0023   
259  CS002515000386         野田 コウ         1     女性  1963-05-30   55  185-0013   
293  CS001615000372        稲垣 寿々花         1     女性  1956-10-29   62  144-0035   
297  CS032512000121         松井 知世         1     女性  1962-09-04   56  210-0011   

                      address application_store_cd application_date  \
12   神奈川県横浜市港北区日吉本町**********        

---
> P-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [35]:
select = df_store[df_store['tel_no'].str.match(r'\d{3}-\d{3}-\d{4}$')]
print(select)

   store_cd store_name prefecture_cd prefecture              address  \
0    S12014       千草台店            12        千葉県      千葉県千葉市稲毛区千草台一丁目   
1    S13002       国分寺店            13        東京都         東京都国分寺市本多二丁目   
2    S14010        菊名店            14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3    S14033       阿久和店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
4    S14036     相模原中央店            14       神奈川県        神奈川県相模原市中央二丁目   
7    S14040       長津田店            14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
9    S14050      阿久和西店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
11   S13052        森野店            13        東京都          東京都町田市森野三丁目   
12   S14028       二ツ橋店            14       神奈川県       神奈川県横浜市瀬谷区二ツ橋町   
16   S14012      本牧和田店            14       神奈川県        神奈川県横浜市中区本牧和田   
18   S14046       北山田店            14       神奈川県     神奈川県横浜市都筑区北山田一丁目   
19   S14022        逗子店            14       神奈川県         神奈川県逗子市逗子一丁目   
20   S14011      日吉本町店            14       神奈川県    神奈川県横浜市港北区日吉本

---
> P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [36]:
sort= df_customer.sort_values(by='birth_day')
print(sort.head(10))


          customer_id customer_name gender_cd gender   birth_day  age  \
18817  CS003813000014        村山 菜々美         1     女性  1928-11-26   90   
12328  CS026813000004         吉村 朝陽         1     女性  1928-12-14   90   
15682  CS018811000003         熊沢 美里         1     女性  1929-01-07   90   
15302  CS027803000004         内村 拓郎         0     男性  1929-01-12   90   
1681   CS013801000003         天野 拓郎         0     男性  1929-01-15   90   
7511   CS001814000022         鶴田 里穂         1     女性  1929-01-28   90   
2378   CS016815000002         山元 美紀         1     女性  1929-02-22   90   
4680   CS009815000003         中田 里穂         1     女性  1929-04-08   89   
16070  CS005813000015        金谷 恵梨香         1     女性  1929-04-09   89   
6305   CS012813000013         宇野 南朋         1     女性  1929-04-09   89   

      postal_cd                 address application_store_cd application_date  \
18817  182-0007     東京都調布市菊野台**********               S13003         20160214   
12328  251-0043   神奈川県藤沢市辻堂元町*****

---
> P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [37]:
sort = df_customer.sort_values(by='birth_day', ascending=True)
print(sort.head(10))


          customer_id customer_name gender_cd gender   birth_day  age  \
18817  CS003813000014        村山 菜々美         1     女性  1928-11-26   90   
12328  CS026813000004         吉村 朝陽         1     女性  1928-12-14   90   
15682  CS018811000003         熊沢 美里         1     女性  1929-01-07   90   
15302  CS027803000004         内村 拓郎         0     男性  1929-01-12   90   
1681   CS013801000003         天野 拓郎         0     男性  1929-01-15   90   
7511   CS001814000022         鶴田 里穂         1     女性  1929-01-28   90   
2378   CS016815000002         山元 美紀         1     女性  1929-02-22   90   
4680   CS009815000003         中田 里穂         1     女性  1929-04-08   89   
16070  CS005813000015        金谷 恵梨香         1     女性  1929-04-09   89   
6305   CS012813000013         宇野 南朋         1     女性  1929-04-09   89   

      postal_cd                 address application_store_cd application_date  \
18817  182-0007     東京都調布市菊野台**********               S13003         20160214   
12328  251-0043   神奈川県藤沢市辻堂元町*****

---
> P-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [38]:
df_receipt['rank'] = df_receipt['amount'].rank(method='dense', ascending=False)
result = df_receipt.sort_values(by='amount', ascending=False).head(10)[['customer_id', 'amount', 'rank']]
print(result)

          customer_id  amount  rank
1202   CS011415000006   10925   1.0
62317  ZZ000000000000    6800   2.0
54095  CS028605000002    5780   3.0
72747  ZZ000000000000    5480   4.0
4632   CS015515000034    5480   4.0
10320  ZZ000000000000    5480   4.0
28304  ZZ000000000000    5440   5.0
97294  CS021515000089    5440   5.0
77874  CS008513000063    5280   6.0
55795  CS032414000072    5280   6.0


---
> P-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [48]:
df_receipt['rank'] = df_receipt['amount'].rank(method='first', ascending=False)
result = df_receipt.sort_values(by='amount', ascending=False).head(10)[['customer_id', 'amount', 'rank']]
print(result)


          customer_id  amount  rank
1202   CS011415000006   10925   1.0
62317  ZZ000000000000    6800   2.0
54095  CS028605000002    5780   3.0
72747  ZZ000000000000    5480   6.0
4632   CS015515000034    5480   4.0
10320  ZZ000000000000    5480   5.0
28304  ZZ000000000000    5440   7.0
97294  CS021515000089    5440   8.0
77874  CS008513000063    5280  31.0
55795  CS032414000072    5280  23.0


---
> P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [51]:
len(df_receipt)

104681

---
> P-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [52]:
df_receipt['customer_id'].nunique()

8307

---
> P-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [53]:
df_receipt.groupby('store_cd').agg({'amount': 'sum', 'quantity': 'sum'}).reset_index()

,store_cd,amount,quantity
0,S12007,638761,2099
1,S12013,787513,2425
2,S12014,725167,2358
3,S12029,794741,2555
4,S12030,684402,2403
5,S13001,811936,2347
6,S13002,727821,2340
7,S13003,764294,2197
8,S13004,779373,2390
9,S13005,629876,2004


---
> P-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [58]:
sales_ymd=df_receipt.groupby('customer_id')['sales_ymd'].max().reset_index()
result=sales_ymd.head(10)
print(result)

      customer_id  sales_ymd
0  CS001113000004   20190308
1  CS001114000005   20190731
2  CS001115000010   20190405
3  CS001205000004   20190625
4  CS001205000006   20190224
5  CS001211000025   20190322
6  CS001212000027   20170127
7  CS001212000031   20180906
8  CS001212000046   20170811
9  CS001212000070   20191018


---
> P-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [60]:
sales_ymd = df_receipt.groupby('customer_id')['sales_ymd'].min().reset_index()
sales_ymd.head(10)

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20180503
2,CS001115000010,20171228
3,CS001205000004,20170914
4,CS001205000006,20180207
5,CS001211000025,20190322
6,CS001212000027,20170127
7,CS001212000031,20180906
8,CS001212000046,20170811
9,CS001212000070,20191018


---
> P-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [63]:
sales = df_receipt.groupby('customer_id')['sales_ymd'].agg(['max', 'min']).reset_index()
different = sales[sales['max'] != sales['min']]
result=different.head(10)
print(result)

       customer_id       max       min
1   CS001114000005  20190731  20180503
2   CS001115000010  20190405  20171228
3   CS001205000004  20190625  20170914
4   CS001205000006  20190224  20180207
13  CS001214000009  20190902  20170306
14  CS001214000017  20191006  20180828
16  CS001214000048  20190929  20171109
17  CS001214000052  20190617  20180208
20  CS001215000005  20181021  20170206
21  CS001215000040  20171022  20170214


---
> P-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [65]:
avg = df_receipt.groupby('store_cd')['amount'].mean().reset_index()
top = avg.sort_values(by='amount', ascending=False).head(5)

print(top)

   store_cd      amount
28   S13052  402.867470
12   S13015  351.111960
7    S13003  350.915519
30   S14010  348.791262
5    S13001  348.470386


---
> P-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [68]:
med = df_receipt.groupby('store_cd')['amount'].median().reset_index()
top = med.sort_values(by='amount', ascending=False).head(5)

print(top)

   store_cd  amount
28   S13052   190.0
30   S14010   188.0
51   S14050   185.0
44   S14040   180.0
7    S13003   180.0


---
> P-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [69]:
mode = df_receipt.groupby('store_cd')['product_cd'].agg(lambda x: x.mode()[0]).reset_index()
result = mode.head(10)
print(result)

  store_cd  product_cd
0   S12007  P060303001
1   S12013  P060303001
2   S12014  P060303001
3   S12029  P060303001
4   S12030  P060303001
5   S13001  P060303001
6   S13002  P060303001
7   S13003  P071401001
8   S13004  P060303001
9   S13005  P040503001


---
> P-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [70]:
var = df_receipt.groupby('store_cd')['amount'].var().reset_index()
top = var.sort_values(by='amount', ascending=False).head(5)

print(top)

   store_cd         amount
28   S13052  441863.252526
31   S14011  306442.242432
42   S14034  297068.392740
5    S13001  295558.842618
12   S13015  295427.197086


---
> P-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

TIPS:

PandasとNumpyでddofのデフォルト値が異なることに注意しましょう
```
Pandas：
DataFrame.std(self, axis=None, skipna=None, level=None, ddof=1, numeric_only=None, **kwargs)
Numpy:
numpy.std(a, axis=None, dtype=None, out=None, ddof=0, keepdims=)
```

In [71]:
sales = df_receipt.groupby('store_cd')['amount'].apply(lambda x: np.std(x, ddof=1)).reset_index()
top = sales.sort_values(by='amount', ascending=False).head(5)
print(top)

   store_cd      amount
28   S13052  664.727954
31   S14011  553.572256
42   S14034  545.039808
5    S13001  543.653237
12   S13015  543.532149


---
> P-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [72]:
percentiles = df_receipt['amount'].quantile([0.25, 0.50, 0.75, 1.0])

print("25%:", percentiles[0.25])
print("50%:", percentiles[0.50])
print("75%:", percentiles[0.75])
print("100%:", percentiles[1.0])

25%: 102.0
50%: 170.0
75%: 288.0
100%: 10925.0


---
> P-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [73]:
avg = df_receipt.groupby('store_cd')['amount'].mean().reset_index()
avg_330 = avg[avg['amount'] >= 330]

print(avg_330)

   store_cd      amount
1    S12013  330.194130
5    S13001  348.470386
7    S13003  350.915519
8    S13004  330.943949
12   S13015  351.111960
16   S13019  330.208616
17   S13020  337.879932
28   S13052  402.867470
30   S14010  348.791262
31   S14011  335.718333
38   S14026  332.340588
46   S14045  330.082073
48   S14047  330.077073


---
> P-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [74]:
sales = df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id')['amount'].sum()
avg = sales.mean()

print(avg)

2547.742234529256


---
> P-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [75]:
sales = df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id')['amount'].sum()
avg = sales.mean()
top=sales[sales >= avg].head(10)
print(top)

customer_id
CS001115000010    3044
CS001205000006    3337
CS001214000009    4685
CS001214000017    4132
CS001214000052    5639
CS001215000040    3496
CS001304000006    3726
CS001305000005    3485
CS001305000011    4370
CS001315000180    3300
Name: amount, dtype: int64


---
> P-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [78]:
merged = pd.merge(df_receipt, df_store, on='store_cd', how='inner')
result =merged[['store_cd','store_name']].head(10)
print(result)

  store_cd store_name
0   S14006       葛が谷店
1   S14006       葛が谷店
2   S14006       葛が谷店
3   S14006       葛が谷店
4   S14006       葛が谷店
5   S14006       葛が谷店
6   S14006       葛が谷店
7   S14006       葛が谷店
8   S14006       葛が谷店
9   S14006       葛が谷店


---
> P-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [80]:
pd.merge(df_product
         , df_category[['category_small_cd','category_small_name']]
         , how='inner', on='category_small_cd').head(10)
#ansを参考にしました

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
0,P040101001,04,0401,040101,198.0,149.0,弁当類
1,P040101002,04,0401,040101,218.0,164.0,弁当類
2,P040101003,04,0401,040101,230.0,173.0,弁当類
3,P040101004,04,0401,040101,248.0,186.0,弁当類
4,P040101005,04,0401,040101,268.0,201.0,弁当類
5,P040101006,04,0401,040101,298.0,224.0,弁当類
6,P040101007,04,0401,040101,338.0,254.0,弁当類
7,P040101008,04,0401,040101,420.0,315.0,弁当類
8,P040101009,04,0401,040101,498.0,374.0,弁当類
9,P040101010,04,0401,040101,580.0,435.0,弁当類


---
> P-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [82]:
woman = df_customer[(df_customer['gender_cd'] == '1') & (~df_customer['customer_id'].str.startswith('Z'))]
customer = df_receipt.groupby('customer_id')['amount'].sum().reset_index()
merged = pd.merge(woman, customer, on='customer_id', how='left')
merged['amount'] = merged['amount'].fillna(0)
result = merged[['customer_id', 'customer_name', 'amount']].head(10)
print(result)


      customer_id customer_name  amount
0  CS021313000114        大野 あや子     0.0
1  CS031415000172       宇多田 貴美子  5088.0
2  CS028811000001        堀井 かおり     0.0
3  CS001215000145         田崎 美紀   875.0
4  CS015414000103         奥野 陽子  3122.0
5  CS033513000180          安斎 遥   868.0
6  CS035614000014        板倉 菜々美     0.0
7  CS011215000048         芦田 沙耶  3444.0
8  CS009413000079         市川 コウ     0.0
9  CS040412000191         川井 郁恵   210.0


---
> P-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [86]:
days = df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id')['sales_ymd'].nunique().reset_index()
days = days.sort_values(by='sales_ymd', ascending=False).head(20)
amount = df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id')['amount'].sum().reset_index()
amount = amount.sort_values(by='amount', ascending=False).head(20)
merged = pd.merge(days, amount, on='customer_id', how='outer')

print(merged)


       customer_id  sales_ymd   amount
0   CS040214000008       23.0      NaN
1   CS015415000185       22.0  20153.0
2   CS010214000010       22.0  18585.0
3   CS010214000002       21.0      NaN
4   CS028415000007       21.0  19127.0
5   CS017415000097       20.0  23086.0
6   CS016415000141       20.0  18372.0
7   CS031414000051       19.0  19202.0
8   CS014214000023       19.0      NaN
9   CS022515000226       19.0      NaN
10  CS021515000172       19.0      NaN
11  CS039414000052       19.0      NaN
12  CS021514000045       19.0      NaN
13  CS022515000028       18.0      NaN
14  CS030214000008       18.0      NaN
15  CS021515000056       18.0      NaN
16  CS014415000077       18.0      NaN
17  CS021515000211       18.0      NaN
18  CS032415000209       18.0      NaN
19  CS031414000073       18.0      NaN
20  CS001605000009        NaN  18925.0
21  CS006515000023        NaN  18372.0
22  CS011414000106        NaN  18338.0
23  CS038415000104        NaN  17847.0
24  CS035414000024       

---
> P-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [87]:
import itertools
product = list(itertools.product(df_store['store_cd'], df_product['product_cd']))
combinations = len(product)

print(combinations)

531590


---
> P-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [5]:

day_sales = df_receipt.groupby('sales_ymd')['amount'].sum().reset_index()
day_sales['prev_amount'] = day_sales['amount'].shift(1)
day_sales['change'] = day_sales['amount'] - day_sales['prev_amount']

# 結果を10件表示（ここではサンプルデータが少ないので全件表示）
print(day_sales.head(10))

   sales_ymd  amount  prev_amount  change
0   20170101   33723          NaN     NaN
1   20170102   24165      33723.0 -9558.0
2   20170103   27503      24165.0  3338.0
3   20170104   36165      27503.0  8662.0
4   20170105   37830      36165.0  1665.0
5   20170106   32387      37830.0 -5443.0
6   20170107   23415      32387.0 -8972.0
7   20170108   24737      23415.0  1322.0
8   20170109   26718      24737.0  1981.0
9   20170110   20143      26718.0 -6575.0


---
> P-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [7]:

day_sales = df_receipt.groupby('sales_ymd')['amount'].sum().reset_index()
day_sales['zenkai'] = day_sales['amount'].shift(1)
day_sales['zenzenkai'] = day_sales['amount'].shift(2)
day_sales['sannkaimae'] = day_sales['amount'].shift(3)
print(day_sales.head(10))


                      sales_ymd  amount   zenkai  zenzenkai  sannkaimae
0 1970-01-01 00:00:00.020170101   33723      NaN        NaN         NaN
1 1970-01-01 00:00:00.020170102   24165  33723.0        NaN         NaN
2 1970-01-01 00:00:00.020170103   27503  24165.0    33723.0         NaN
3 1970-01-01 00:00:00.020170104   36165  27503.0    24165.0     33723.0
4 1970-01-01 00:00:00.020170105   37830  36165.0    27503.0     24165.0
5 1970-01-01 00:00:00.020170106   32387  37830.0    36165.0     27503.0
6 1970-01-01 00:00:00.020170107   23415  32387.0    37830.0     36165.0
7 1970-01-01 00:00:00.020170108   24737  23415.0    32387.0     37830.0
8 1970-01-01 00:00:00.020170109   26718  24737.0    23415.0     32387.0
9 1970-01-01 00:00:00.020170110   20143  26718.0    24737.0     23415.0


---
> P-043： レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [11]:

df_customer['nenndai'] = (df_customer['age'] // 10) * 10
df_merged = pd.merge(df_receipt, df_customer, on='customer_id', how='inner')
summary = df_merged.groupby(['nenndai', 'gender_cd'])['amount'].sum().unstack(fill_value=0)
summary = summary.rename(columns={0: 'male_sales', 1: 'female_sales', 9: 'unknown_sales'}).reset_index()
print(summary)


gender_cd  nenndai       0        1       9
0               10    1591   149836    4317
1               20   72940  1363724   44328
2               30  177322   693047   50441
3               40   19355  9320791  483512
4               50   54320  6685192  342923
5               60  272469   987741   71418
6               70   13435    29764    2427
7               80   46360   262923    5111
8               90       0     6260       0


---
> P-044： 043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [15]:
df_melted = summary.melt(id_vars='nenndai', 
                                  value_vars=['male_sales', 'female_sales', 'unknown_sales'], 
                                  var_name='gender_cd', 
                                  value_name='amount')

df_melted['gender_cd'] = df_melted['gender_cd'].map({
    'male_sales': '00',
    'female_sales': '01',
    'unknown_sales': '99'
})

print(df_melted)

KeyError: "The following 'value_vars' are not present in the DataFrame: ['female_sales', 'male_sales', 'unknown_sales']"

---
> P-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [19]:
df_customer['birth_day'] = pd.to_datetime(df_customer['birth_day'])
df_customer['birth_day'] = df_customer['birth_day'].dt.strftime('%Y%m%d')
print(df_customer[['customer_id', 'birth_day']].head(10))

      customer_id birth_day
0  CS021313000114  19810429
1  CS037613000071  19520401
2  CS031415000172  19761004
3  CS028811000001  19330327
4  CS001215000145  19950329
5  CS020401000016  19740915
6  CS015414000103  19770809
7  CS029403000008  19730817
8  CS015804000004  19310502
9  CS033513000180  19620711


---
> P-046: 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [20]:
df_customer['application_date'] = pd.to_datetime(df_customer['application_date'], format='%Y%m%d')
print(df_customer[['customer_id', 'application_date']].head(10))

      customer_id application_date
0  CS021313000114       2015-09-05
1  CS037613000071       2015-04-14
2  CS031415000172       2015-05-29
3  CS028811000001       2016-01-15
4  CS001215000145       2017-06-05
5  CS020401000016       2015-02-25
6  CS015414000103       2015-07-22
7  CS029403000008       2015-05-15
8  CS015804000004       2015-06-07
9  CS033513000180       2015-07-28


---
> P-047: レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [21]:
df_receipt['sales_ymd'] = pd.to_datetime(df_receipt['sales_ymd'], format='%Y%m%d')
print(df_receipt[['receipt_no', 'receipt_sub_no', 'sales_ymd']].head(10))

   receipt_no  receipt_sub_no                     sales_ymd
0         112               1 1970-01-01 00:00:00.020181103
1        1132               2 1970-01-01 00:00:00.020181118
2        1102               1 1970-01-01 00:00:00.020170712
3        1132               1 1970-01-01 00:00:00.020190205
4        1102               2 1970-01-01 00:00:00.020180821
5        1112               1 1970-01-01 00:00:00.020190605
6        1102               2 1970-01-01 00:00:00.020181205
7        1102               1 1970-01-01 00:00:00.020190922
8        1112               2 1970-01-01 00:00:00.020170504
9        1102               1 1970-01-01 00:00:00.020191010


---
> P-048: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [22]:
df_receipt['sales_datetime'] = pd.to_datetime(df_receipt['sales_epoch'], unit='s')
print(df_receipt[['receipt_no', 'receipt_sub_no', 'sales_datetime']].head(10))

   receipt_no  receipt_sub_no sales_datetime
0         112               1     2018-11-03
1        1132               2     2018-11-18
2        1102               1     2017-07-12
3        1132               1     2019-02-05
4        1102               2     2018-08-21
5        1112               1     2019-06-05
6        1102               2     2018-12-05
7        1102               1     2019-09-22
8        1112               2     2017-05-04
9        1102               1     2019-10-10


---
> P-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [23]:
df_receipt['sales_datetime'] = pd.to_datetime(df_receipt['sales_epoch'], unit='s')
df_receipt['sales_year'] = df_receipt['sales_datetime'].dt.year
print(df_receipt[['receipt_no', 'receipt_sub_no', 'sales_year']].head(10))

   receipt_no  receipt_sub_no  sales_year
0         112               1        2018
1        1132               2        2018
2        1102               1        2017
3        1132               1        2019
4        1102               2        2018
5        1112               1        2019
6        1102               2        2018
7        1102               1        2019
8        1112               2        2017
9        1102               1        2019


---
> P-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [24]:
df_receipt['sales_datetime'] = pd.to_datetime(df_receipt['sales_epoch'], unit='s')
df_receipt['sales_month'] = df_receipt['sales_datetime'].dt.strftime('%m')
print(df_receipt[['receipt_no', 'receipt_sub_no', 'sales_month']].head(10))

   receipt_no  receipt_sub_no sales_month
0         112               1          11
1        1132               2          11
2        1102               1          07
3        1132               1          02
4        1102               2          08
5        1112               1          06
6        1102               2          12
7        1102               1          09
8        1112               2          05
9        1102               1          10


---
> P-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [25]:
df_receipt['sales_datetime'] = pd.to_datetime(df_receipt['sales_epoch'], unit='s')
df_receipt['sales_day'] = df_receipt['sales_datetime'].dt.strftime('%d')
print(df_receipt[['receipt_no', 'receipt_sub_no', 'sales_day']].head(10))

   receipt_no  receipt_sub_no sales_day
0         112               1        03
1        1132               2        18
2        1102               1        12
3        1132               1        05
4        1102               2        21
5        1112               1        05
6        1102               2        05
7        1102               1        22
8        1112               2        04
9        1102               1        10


---
> P-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [27]:
df_receipt = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]
df_customer_sales = df_receipt.groupby('customer_id')['amount'].sum().reset_index()
df_customer_sales['2000<'] = (df_customer_sales['amount'] > 2000).astype(int)
print(df_customer_sales.head(10))

      customer_id  amount  2000<
0  CS001113000004    1298      0
1  CS001114000005     626      0
2  CS001115000010    3044      1
3  CS001205000004    1988      0
4  CS001205000006    3337      1
5  CS001211000025     456      0
6  CS001212000027     448      0
7  CS001212000031     296      0
8  CS001212000046     228      0
9  CS001212000070     456      0


---
> P-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [29]:
df_customer['tokyo'] = df_customer['postal_cd'].str[:3].astype(int).between(100, 209).astype(int)
df_merged = pd.merge(df_customer, df_receipt, on='customer_id', how='inner')
customer_count_by_tokyo = df_merged.groupby('is_tokyo')['customer_id'].nunique()
print(customer_count_by_tokyo)

is_tokyo
0    3906
1    4400
Name: customer_id, dtype: int64


---
> P-054: 顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [31]:
def get_prefecture_code(address):
    if '埼玉県' in address:
        return 11
    elif '千葉県' in address:
        return 12
    elif '東京都' in address:
        return 13
    elif '神奈川県' in address:
        return 14
    else:
        return None 
df_customer['prefecture_code'] = df_customer['address'].apply(get_prefecture_code)
print(df_customer[['customer_id', 'address', 'prefecture_code']].head(10))

      customer_id                 address  prefecture_code
0  CS021313000114    神奈川県伊勢原市粟窪**********               14
1  CS037613000071      東京都江東区南砂**********               13
2  CS031415000172     東京都渋谷区代々木**********               13
3  CS028811000001  神奈川県横浜市泉区和泉町**********               14
4  CS001215000145     東京都大田区仲六郷**********               13
5  CS020401000016      東京都板橋区若木**********               13
6  CS015414000103      東京都江東区北砂**********               13
7  CS029403000008      千葉県浦安市海楽**********               12
8  CS015804000004      東京都江東区北砂**********               13
9  CS033513000180  神奈川県横浜市旭区善部町**********               14


---
> P-055: レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [32]:
df_customer_sales = df_receipt.groupby('customer_id')['amount'].sum()
quartiles = np.percentile(df_customer_sales, [25, 50, 75])
def categorize_amount(total_amount, quartiles):
    if total_amount < quartiles[0]:
        return 1
    elif total_amount < quartiles[1]:
        return 2
    elif total_amount < quartiles[2]:
        return 3
    else:
        return 4
df_customer_sales['category'] = df_customer_sales.apply(categorize_amount, args=(quartiles,))
print(df_customer_sales.head(10))

customer_id
CS001113000004    1298
CS001114000005     626
CS001115000010    3044
CS001205000004    1988
CS001205000006    3337
CS001211000025     456
CS001212000027     448
CS001212000031     296
CS001212000046     228
CS001212000070     456
Name: amount, dtype: object


---
> P-056: 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [33]:
def categorize_age(age):
    if age >= 60:
        return '60歳代'
    else:
        decade = (age // 10) * 10
        return f'{decade}代'
df_customer['generation'] = df_customer['age'].apply(categorize_age)
print(df_customer[['customer_id', 'birth_day', 'generation']].head(10))

      customer_id birth_day generation
0  CS021313000114  19810429        30代
1  CS037613000071  19520401       60歳代
2  CS031415000172  19761004        40代
3  CS028811000001  19330327       60歳代
4  CS001215000145  19950329        20代
5  CS020401000016  19740915        40代
6  CS015414000103  19770809        40代
7  CS029403000008  19730817        40代
8  CS015804000004  19310502       60歳代
9  CS033513000180  19620711        50代


---
> P-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [34]:
def categorize_gender_cd(gender_cd):
    if gender_cd == 1:
        return '男性'
    elif gender_cd == 2:
        return '女性'
    else:
        return '不明'
df_customer['gender_generation'] = df_customer['gender_cd'].apply(categorize_gender_cd) + '_' + df_customer['generation']
print(df_customer[['customer_id', 'birth_day', 'gender_generation']].head(10))

      customer_id birth_day gender_generation
0  CS021313000114  19810429            不明_30代
1  CS037613000071  19520401           不明_60歳代
2  CS031415000172  19761004            不明_40代
3  CS028811000001  19330327           不明_60歳代
4  CS001215000145  19950329            不明_20代
5  CS020401000016  19740915            不明_40代
6  CS015414000103  19770809            不明_40代
7  CS029403000008  19730817            不明_40代
8  CS015804000004  19310502           不明_60歳代
9  CS033513000180  19620711            不明_50代


---
> P-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [35]:
df_gender_dummy = pd.get_dummies(df_customer['gender_cd'], prefix='gender_cd')
df_result = pd.concat([df_customer['customer_id'], df_gender_dummy], axis=1)
print(df_result.head(10))

      customer_id  gender_cd_0  gender_cd_1  gender_cd_9
0  CS021313000114            0            1            0
1  CS037613000071            0            0            1
2  CS031415000172            0            1            0
3  CS028811000001            0            1            0
4  CS001215000145            0            1            0
5  CS020401000016            1            0            0
6  CS015414000103            0            1            0
7  CS029403000008            1            0            0
8  CS015804000004            1            0            0
9  CS033513000180            0            1            0


---
> P-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

TIPS:
- query()の引数engineで'python'か'numexpr'かを選択でき、デフォルトはインストールされていればnumexprが、無ければpythonが使われます。さらに、文字列メソッドはengine='python'でないとquery()内で使えません。


In [36]:
df_receipt = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]
df_sales_total = df_receipt.groupby('customer_id')['amount'].sum().reset_index()
sales_total_mean = df_sales_total['amount'].mean()
sales_total_std = df_sales_total['amount'].std()
df_sales_total['amount_standardized'] = (df_sales_total['amount'] - sales_total_mean) / sales_total_std
print(df_sales_total[['customer_id', 'amount_standardized']].head(10))

      customer_id  amount_standardized
0  CS001113000004            -0.459350
1  CS001114000005            -0.706348
2  CS001115000010             0.182403
3  CS001205000004            -0.205737
4  CS001205000006             0.290096
5  CS001211000025            -0.768832
6  CS001212000027            -0.771773
7  CS001212000031            -0.827641
8  CS001212000046            -0.852635
9  CS001212000070            -0.768832


---
> P-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [38]:
df_receipt = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]
df_sales_total = df_receipt.groupby('customer_id')['amount'].sum().reset_index()
min_amount = df_sales_total['amount'].min()
max_amount = df_sales_total['amount'].max()
df_sales_total['amount_normalized'] = (df_sales_total['amount'] - min_amount) / (max_amount - min_amount)
print(df_sales_total[['customer_id', 'amount_normalized']].head(10))

      customer_id  amount_normalized
0  CS001113000004           0.053354
1  CS001114000005           0.024157
2  CS001115000010           0.129214
3  CS001205000004           0.083333
4  CS001205000006           0.141945
5  CS001211000025           0.016771
6  CS001212000027           0.016423
7  CS001212000031           0.009819
8  CS001212000046           0.006865
9  CS001212000070           0.016771


---
> P-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

---
> P-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

---
> P-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

---
> P-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

---
> P-065: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

---
> P-066: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

---
> P-067: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

---
> P-068: 商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

---
> P-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

---
> P-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

---
> P-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

---
> P-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

---
> P-073: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

---
> P-074: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

---
> P-075: 顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

---
> P-076: 顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

---
> P-077: レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

---
> P-078: レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

---
> P-079: 商品データ（df_product）の各項目に対し、欠損数を確認せよ。

---
> P-080: 商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-084: 顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

---
> P-085: 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

---
> P-086: 085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

$$
\mbox{緯度（ラジアン）}：\phi \\
\mbox{経度（ラジアン）}：\lambda \\
\mbox{距離}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2
+ \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
$$

---
> P-087: 顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

---
> P-088: 087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
> 
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

---
> P-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

---
> P-090: レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

---
> P-091: 顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

---
> P-092: 顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

---
> P-093: 商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

---
> P-094: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

---
> P-095: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

---
> P-096: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

---
> P-097: 094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

---
> P-098: 096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

---
> P-099: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

---
> P-100: 099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

# これで１００本終わりです。おつかれさまでした！